In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import re
from collections import Counter

In [3]:
class ArhivarijParser(object):
    def __init__(self):
        self.link = "http://arhivarij.narod.ru/"
        self.links = []
        self.alphabeth = 'í́ІАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяіѢѣѲѳѴѵ'
        self.restrict = 0
        self.allarticles = ""

    def _get_links(self, link):
        
        while True:
            try:
                resp = requests.get(link)
                bss = BeautifulSoup(resp.text, "html5lib")
                break
                
            except:
                print('failed')
                self.restrict+=1
                if self.restrict>200:
                    break
                
        try:
            for a in bss.find_all("a"):
                try:
                    start_link = str(a).find('http://')
                    end_link = str(a).find('.html')+5
                    link = str(a)[start_link:end_link]
                    if link not in self.links:
                        self.links.append(link)
                except:
                    print('Что-то не так с получением ссылок')

                print('Внимание! Обнаружено ', len(self.links), ' статей!\r', end='')
        except:
            print('Видимо не нашлась ссылка')
        print()

    def get_links(self):
        
        print('Обрабатываю ')
        
        self._get_links(self.link)
        
        print('Обрабатываю еще')
        for link in self.links:
            self._get_links(link)
            if len(self.links) >3000:
                break

    def _get_article(self, link):
        """
        Парсим тело статьи
        """
        resp = requests.get(link)
        bs = BeautifulSoup(resp.text, "html5lib")
        raw_article = ' '.join([re.sub('\s+',
                                   ' ',
                                   re.sub(r"[^0-9í́ІIАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяіiѢѣѲѳѴѵ.,]",
                                          ' ',
                                          str(e))).strip() for e in bs()])
#         print('got raw article')
        counts=Counter(list(raw_article))
#         print('got counts')
        i_counts = counts['i']
        I_counts = counts['I']
#         print('counts of i = {}\ncounts of I = {}'.format(i_counts, I_counts))
        
        for  e in range(i_counts):
            if raw_article[raw_article.find('i')+1] in self.alphabeth or raw_article[raw_article.find('i')+1] in self.alphabeth:
                raw_article=raw_article[:raw_article.find('i')]+'і'+raw_article[raw_article.find('i')+1:]
            else: 
                raw_article=raw_article[:raw_article.find('i')]+raw_article[raw_article.find('i')+1:]
                
        raw_article = raw_article.replace(' .', '')
        
        for  e in range(I_counts):
            if raw_article[raw_article.find('I')+1] in self.alphabeth or raw_article[raw_article.find('I')+1] in self.alphabeth:
                raw_article=raw_article[:raw_article.find('I')]+'І'+raw_article[raw_article.find('I')+1:]
            else: 
                raw_article=raw_article[:raw_article.find('I')]+raw_article[raw_article.find('I')+1:]
        article = re.sub('\s+',' ',raw_article)
        return article.replace(' ,', ',').replace(',,', ',')

    def get_all_articles(self):
        """
        Проходим по всем статьям и парсим их
        """
        links = []  # список статей
        count = 0  # это показатель, сколько статей обработано (т.к. все достаточно медленно)
        success = 0
        for link in self.links:
            lk = link
            try:
                self.allarticles += self._get_article(lk)
                success += 1
            except:
                print("didn't get article")
                pass
            count += 1
            if success>360:
                break
            print('Обработано статей: {} Успешно: {}'.format(count, success))

    
    def to_txt(self):
        """
        Записываем в файл
        """
        with open('corpus3.txt', 'a', encoding='utf-8-sig') as f:
            f.write(self.allarticles)

In [205]:
'I'=='І'

False

In [1]:
class RusPortalParser(object):
    def __init__(self):
        self.link = "http://www.russportal.ru/"
        self.links = []
        self.restrict = 0

    def _get_links(self, link):

        while True:
            try:
                resp = requests.get(link)
                bss = BeautifulSoup(resp.text, "html5lib")
                break
            except:
                print('failed')
                self.restrict+=1
                if self.restrict>50:
                    break
        try:
            for element in bss.find_all("div"):
                try:
                    link = element.find("a")["href"]
                    if 'http' in link:
                        if link not in self.links:
                            self.links.append(link)
                except:
                    pass

                self.links.append(link)
                print('Внимание! Обнаружено ', len(self.links), ' статей!\r', end='')
            print()
        except:
            print('Видимо не нашлась ссылка')

    def get_links(self):

        print('Обрабатываю ')
        self._get_links(self.link)
        for lk in self.links:
            self._get_links(lk)
            if len(self.links) >20000:
                break

    def _get_article(self, link):
        """
        Парсим тело статьи
        """
        resp = requests.get(link)
        bs = BeautifulSoup(resp.text, "html5lib")
        article = ' '.join([re.sub('\s+',
                                   ' ', 
                                   re.sub(r"[^0-9í́ІАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяіѢѣѲѳѴѵ.,-]", 
                                          ' ',
                                          str(e))).strip() for e in bs('p')])
        return article

    def get_all_articles(self):
        """
        Проходим по всем статьям и парсим их
        """
        links = []  # список статей
        self.allarticles = ""
        count = 0  # это показатель, сколько статей обработано (т.к. все достаточно медленно)
        success = 0
        for link in self.links:
            lk = link
            try:
                self.allarticles += re.sub('\s+',' ', self._get_article(lk))
                success += 1
            except:
                pass
            count += 1
            if success>10000:
                break
            print('Обработано статей:', count, ' Успешно:', success, '\r', end='')

    
    def to_txt(self):
        """
        Записываем в файл
        """
        with open('corpus2.txt', 'a', encoding='utf-8-sig') as f:
            f.write(self.allarticles)

In [4]:
my = RusPortalParser()

In [4]:
A = ArhivarijParser()

In [ ]:
%%time
my.get_links()

In [ ]:
%%time
A.get_links()

In [6]:
%%time
my.get_all_articles()

CPU times: user 3.53 s, sys: 1.7 s, total: 5.23 s
Wall time: 1h 18min 17s


In [ ]:
# 183
# 4871
# 2990000

In [ ]:
%%time
A.get_all_articles()

In [16]:
len(re.sub('\s+',' ',my.allarticles).split())

16282486

In [17]:
%%time
my.to_txt()

CPU times: user 766 ms, sys: 1.22 s, total: 1.98 s
Wall time: 2.4 s


In [9]:
%%time
A.to_txt()

CPU times: user 125 ms, sys: 125 ms, total: 250 ms
Wall time: 531 ms
